In [40]:
import numpy as np
import pandas as pd

from tensorflow.keras import Sequential, layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from matches_clean import clean_matches_data, clean_player_data

## Data preprocessing

### Retrieve data and merge

In [4]:
matches = clean_matches_data()
players = clean_player_data()

/root/code/jasminebz/1039_esports/1039_esports/matches_clean.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["obs_placed"] = df_clean["obs_placed"].replace(np.nan, 0)
/root/code/jasminebz/1039_esports/1039_esports/matches_clean.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["roshans_killed"] = df_clean["roshans_killed"].replace(np.nan, 0)


In [5]:
df = matches.merge(players, on= "match_id")

### encode y

In [6]:
X = df.drop(columns = ["match_id", "radiant_win", "hero_id", "account_id", "isRadiant", "win"])
y = df["hero_id"]

In [28]:
X.shape

(1358, 15)

In [47]:
y.unique()

array([ 18,  67,  87,  32,   8, 104,  10,  40, 136,  54, 135,   5,   9,
       100,  95,  37,  99,  50,  85, 112,  28,   6, 126,  71,  86,  14,
        94,  31,  33,  93,  41,  82,  76,  19,  53,  47,  35, 114, 123,
        84,  38, 106,  46,  72,  70,  80,  75, 102, 119,  11,  52, 108,
        17, 129, 110, 105,  90, 101,  64, 121,  44,  58,  56,   7,  43,
        59,  22,   1, 128,  26,  12,  20,   2,  62,  88,  79,   4,  16,
        39,  30,  42,  74,  51,  45,  27,  97,  21,  15,  89,  92, 113,
        29,  25, 120, 107,  36,  23,  69,  48,  57,  81,  60,  96,  61,
        73, 137, 103,  55,  98, 111,  68,  34,  13,   3,  83, 109,  78,
        66,  63,  91,  49])

In [34]:
y_cat = to_categorical(y)

In [36]:
y_cat.shape

(1358, 138)

In [46]:
pd.DataFrame(y_cat).sum()

0       0.0
1      17.0
2      23.0
3       2.0
4       8.0
       ... 
133     0.0
134     0.0
135    16.0
136    26.0
137     6.0
Length: 138, dtype: float32

### Train-test-split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat)

### Scaling

In [15]:
scaler = StandardScaler()

In [16]:
X_train = scaler.fit_transform(X_train)

In [18]:
X_test =-scaler.fit_transform(X_test)

## Model

In [21]:
def init_model():
    
    # model architecture
    model = Sequential()
    model.add(layers.Dense(20, activation = "relu", input_dim = 15))
    model.add(layers.Dense(121, activation = "softmax"))
    
    # compiler
    model.compile(loss= "categorical_crossentropy", 
                 optimizer = "adam", 
                 metrics = ["accuracy"])
    
    return model

In [23]:
model = init_model()

In [26]:
es = EarlyStopping(patience = 3, restore_best_weights = True)

In [27]:
history = model.fit(X_train, y_train, callbacks = es, epochs = 50)

Epoch 1/50


ValueError: in user code:

    File "/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/root/.pyenv/versions/3.8.12/envs/lewa